## Introducing the Community Earth System Model (CESM)
This notebook is adapted from [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook) by [Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany.

## About the CESM

### Overview

The CESM is one example of complex coupled GCMS utilized by the Intergovernmental Panel for Climate Change report process.  It is developed and maintained at NCAR by a group of software engineers and climate processes.  The code is open-source, with new pieces contributed by a wide variety of users.

### Key Componenets

The CESM is a modular piece of software, meaning that a researcher can use which submodels to combine together to best answer their invididual research question with the computer power they have available.

Pieces of the CSEM include:
- Atmospheric model (AGCM): Community Atmosphere Model (CAM)
    -- This model maintains a resolution of approximately 2º lat/long, meaning that while it resolves synoptic-scale dynamics like storm tracks and cyclones, it is not capable of resolving mesoscale and smaller events like thunderstorms or clouds.  The model conserves momentum, mass, energy, water, and the equation of state, while parameterizing weather phenomena and solving equations of radiative transfer based on the composition of the atmosphere, absorption properties of different gases, and the radiative effects of clouds
- Ocean model (OGCM): Parallel Ocean Program (POP)
    -- This model has a 1º lat/long resolution and models the exchanges of heat, water, and momentum in the atmosphere and sea ice.  It includes a full 3D simulation of the currents.
- Land surface model: Community Land Model (CLM)
    -- Like the atmospheric model, this model maintains a 2º lat/long resolution.  It's complex, but, in short, determined the surface fluxes of heat, water, and momentum based on vegetation types.
- Sea ice model: Community Ice CodE (CICE)
    -- This model has a 1º lat/long resolution.  It is incredibly complex and uses thermodynamics to determine freezing and melting and momentum equations to track ice motion and deformation.

### Setting up the model
In order to run the model, we need to give it realistic atmospheric compositions, solar radiations, et cetera.  We will first perform a control - a preindustrial run - and get the average of several years, then do our test run - with double the original atmospheric CO2 - and run it several times to allow the model to adjust to the new equilibrium.

All of our data is stored in `NetCDF` files, which contain self-describing gridded data 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

### Boundary conditions: continents and topography

To begin setting up our models, we are first going to input the topography file.  This file was retrieved from the [class Slack page](https://app.slack.com/client/T01JVA6NV46/C01JVA6PJCE) and saved to each of our computers, meaning that the user of this lab write-up will need to update the below file path to retrieve the file.

In [ ]:
#  Open the topography dataset
#  Remember to update file path to the location of your file
topo = xr.open_dataset( "/Users/Andrea/Downloads/USGS-gtopo30_1.9x2.5_remap_c050602.nc" )
print(topo)

# access individual variables within the 'xarray.Dataset'
topo.PHIS

### Plotting the topography
After investigating the topography dataset, we are ready to plot the topography of the Earth's surface on this 2º lat/long grid based on the geopotential variable.  

The below code makes a colorful topographic plot, with a land-scea mask so that nothing is plotted on ocean-covered plot points.  

In [ ]:
g = 9.8  # gravity in m/s2
meters_per_kilometer = 1E3 # define that there are 1000 meters per kilometer
height = topo.PHIS / g / meters_per_kilometer  # convert the topography from meters to kilometers
#  Note that we have just created a new xarray.DataArray object that preserves the axis labels
#  Let's go ahead and give it some useful metadata:
height.attrs['units'] = 'km'
height.name = 'height'
height # open xarray.Dataarray height variable
height.plot() # produce topography plot

To produce a more attractive plot, we can use matplotlib.

In [ ]:
#  A filled contour plot of topography with contours every 500 m
lev = np.arange(0., 6., 0.5)
fig1, ax1 = plt.subplots(figsize=(8,4))
# Here we are masking the data to exclude points where the land fraction is zero (water only)
cax1 = ax1.contourf( height.lon, height.lat, 
                    height.where(topo.LANDFRAC>0), levels=lev)
ax1.set_title('Topography (km) and land-sea mask in CESM')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
cbar1 = fig1.colorbar(cax1)

The 2º of our topography plots means that while we can see certain smaller features like Pacific island, these are lumped together into larger plots with fractional land covery for each grid point.

In order to see areas with only "some" water, we can plot the land-sea mask itself.  Areas with 0.0 ocean mask are completely water (like the middle of an ocean) while areas with 1.0 ocean mask are dry land, like the middle of a contient.  Notice that major likes, like Lake Victoria or Lake Eerie, are not visible on this map.

In [ ]:
fig2, ax2 = plt.subplots() # produce a second figure
cax2 = ax2.pcolormesh( topo.lon, topo.lat, topo.LANDFRAC ) #plot longitude on the x-axis, latitude on the y-axis, and a colorscale
ax2.set_title('Ocean mask in CESM') # title 
ax2.set_xlabel('Longitude'); ax2.set_ylabel('Latitude') # x and y axis labels
cbar2 = fig2.colorbar(cax2); # color key

However, notice that the latitude-longitude array, as plotted above, does not have a map projection and is therefore highly distorted.

### Ocean Boundary Conditions

this is where andrea left off :)